<h1 style='color:#008'>Read all BOS csv insert data on database</h1>

In [1]:
%%time
import pandas as pd
import funcs_bos, gc

files_names = funcs_bos.get_files_names(only_not_imported=False)
print('files ok')
files_names

bos = funcs_bos.read_files(files_names)
print('files lidos ok')

bos = funcs_bos.data_bos_processing(bos)
print('processing df ok')

classificadores = funcs_bos.get_classificadores()
bos['SETOR'] = bos.apply(lambda row: funcs_bos.classifica_setor(row, classificadores), axis=1)
print('classificação setores ok')

funcs_bos.classifica_cia(df_bos=bos)
print('classificação cias ok')

bos.to_sql('tbl_bos', 'sqlite:///gdo.db', if_exists='replace', index=False)
print('dados exportados para o db ok')

del bos
print('bos deleted')
gc.collect()

pd.DataFrame(files_names).to_sql('tbl_imported_files_bos', 'sqlite:///gdo.db', if_exists='replace', index=False)
print('nomes dos arquivos importados, já inseridos no db')

files ok


/home/p323bpm/.asdf/installs/python/anaconda3-2020.02/lib/python3.7/site-packages/IPython/core/magic.py:187: DtypeWarning: Columns (24) have mixed types.Specify dtype option on import or set low_memory=False.
  call = lambda f, *a, **k: f(*a, **k)


files lidos ok
processing df ok
classificação setores ok
classificação cias ok


/home/p323bpm/.asdf/installs/python/anaconda3-2020.02/lib/python3.7/site-packages/pandas/core/generic.py:2663: UserWarning: the 'timedelta' type is not supported, and will be written as integer values (ns frequency) to the database.
  method=method,


dados exportados para o db ok
bos deleted
nomes dos arquivos importados, já inseridos no db
CPU times: user 13.5 s, sys: 439 ms, total: 14 s
Wall time: 14.8 s


In [2]:
bos = pd.read_sql('select * from tbl_bos', 'sqlite:///gdo.db')

In [3]:
reunioes = bos[
    ( bos['ANO'] == 2020 )
    & ( bos['NAT.CODIGO'].str.contains('A19') )
].groupby(['CIA', 'MES']).count()['RAT.NUM_ATIVIDADE'].unstack('MES')
reunioes.loc['TOTAL'] = reunioes.sum()
reunioes.loc[:,'ACUM'] = reunioes.sum(1)
reunioes.to_excel('reunioes.xlsx')

In [4]:
table_bos = dict()
table_bos['mes'] = bos[
    ( bos['MES'] == 6 )
    & ( bos['NAT.CODIGO'].str.contains('A19') )
    & ( bos['ANO'] == 2020 )
].groupby('CIA').count()['RAT.NUM_ATIVIDADE']
table_bos['mes'].loc['TOTAL'] = table_bos['mes'].sum()
table_bos['mes']

table_bos['acum'] = bos[
    ( bos['MES'] <= 4 )
    & ( bos['NAT.CODIGO'].str.contains('A19') )
    & ( bos['ANO'] == 2020 )
].groupby('CIA').count()['RAT.NUM_ATIVIDADE']
table_bos['acum'].loc['TOTAL'] = table_bos['acum'].sum()
table_bos['acum']

CIA
139 CIA     58
142 CIA     94
51 CIA      91
53 CIA      50
other        1
TOTAL      294
Name: RAT.NUM_ATIVIDADE, dtype: int64

In [5]:
bos[
    ( bos['NAT.CODIGO'].str.contains('A19') )
    & ( bos['SETOR'] == 'other' )
][[
    'RAT.NUM_ATIVIDADE',
    'MUNICIPIO',
    'LOGRADOURO',
    'DES_ENDERECO',
    'COMPLEMENTO_ENDERECO',
    'NOME_BAIRRO',
    'LOGRADOURO2',
    'DES_ENDERECO2'
]]

,RAT.NUM_ATIVIDADE,MUNICIPIO,LOGRADOURO,DES_ENDERECO,COMPLEMENTO_ENDERECO,NOME_BAIRRO,LOGRADOURO2,DES_ENDERECO2
4755,2020-014269725-001,DIVINOPOLIS,PERNAMBUCO,PERNAMBUCO,,,,
7940,2020-021182674-001,DIVINOPOLIS,,DULPHE PINTO DE AGUIAR,,,,
7982,2020-021323080-001,DIVINOPOLIS,A,A,,,,
9189,2020-025629373-001,DIVINOPOLIS,PERNAMBUCO,PERNAMBUCO,,,,
